# Stimulus properties

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scipy.ndimage import gaussian_filter
from tqdm.auto import tqdm

import metadata
from load import load_stimulus_movie
from metadata import STIMULUS_METADATA
from spectral_differentiation import join_axes, spectral_differentiation

In [2]:
OUTPUT_DIR = Path('results')

## Load stimuli

In [3]:
stimuli = Parallel(n_jobs=len(STIMULUS_METADATA), verbose=5)(
    delayed(load_stimulus_movie)(stimulus) for stimulus in STIMULUS_METADATA.index
)
stimuli = dict(zip(STIMULUS_METADATA.index, stimuli))

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   2 out of  14 | elapsed:    2.1s remaining:   12.7s
[Parallel(n_jobs=14)]: Done   5 out of  14 | elapsed:    3.5s remaining:    6.3s
[Parallel(n_jobs=14)]: Done   8 out of  14 | elapsed:    5.8s remaining:    4.4s
[Parallel(n_jobs=14)]: Done  11 out of  14 | elapsed:    8.2s remaining:    2.2s
[Parallel(n_jobs=14)]: Done  14 out of  14 | elapsed:   10.6s remaining:    0.0s
[Parallel(n_jobs=14)]: Done  14 out of  14 | elapsed:   10.6s finished


### Blur stimuli by mouse V1 RF size

Calculate std. dev. in pixels of Gaussian blur with half-width at half maximum set to the size of a Cux2 V1 RF

In [4]:
# Median Cux2 V1 RF size from de Vries et al. 2020
RF_AREA = 250  # degrees^2
RF_RADIUS = np.sqrt(RF_AREA / np.pi)  # degrees
# Visual degrees spanned by screen
DEGREES = pd.Series({'x': 120, 'y': 95})
# Stimulus dimensions
PIXELS = pd.Series({'x': 192, 'y': 120})
# Pixel size of RF
RF_PIXELS = RF_RADIUS * PIXELS / DEGREES
# Set half-width at half maximum to RF radius in terms of std. dev.
SIGMA = RF_PIXELS / np.sqrt(2 * np.log(2))

In [5]:
RF_RADIUS

8.920620580763856

In [6]:
RF_PIXELS

x    14.272993
y    11.268152
dtype: float64

In [7]:
def blur(stimulus):
    if stimulus is None:
        return None
    # Apply gaussian filter to each frame (time is first dimension)
    return np.array(
        [
            gaussian_filter(
                img,
                sigma=(SIGMA["y"], SIGMA["x"]),
                order=0,
                mode="reflect",
                truncate=4.0,
            )
            for img in stimulus
        ]
    )

In [8]:
names, movies = list(zip(*stimuli.items()))
blurred = Parallel(n_jobs=len(stimuli), verbose=5)(
    delayed(blur)(movie) for movie in movies
)
blurred_stimuli = dict(zip(names, blurred))

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   2 out of  14 | elapsed:    2.5s remaining:   14.8s
[Parallel(n_jobs=14)]: Done   5 out of  14 | elapsed:    6.5s remaining:   11.6s
[Parallel(n_jobs=14)]: Done   8 out of  14 | elapsed:    8.8s remaining:    6.6s
[Parallel(n_jobs=14)]: Done  11 out of  14 | elapsed:   11.2s remaining:    3.1s
[Parallel(n_jobs=14)]: Done  14 out of  14 | elapsed:   13.7s remaining:    0.0s
[Parallel(n_jobs=14)]: Done  14 out of  14 | elapsed:   13.7s finished


## Helper functions

### Differentiation

In [9]:
def reshape_movie(movie):
    """Reshape a stimulus from (time, x, y) to (trial, cell, sample), where each pixel is a 'cell'."""
    data = join_axes(1, 2, movie)
    # Move time to the last axis (assumes time is the first axis)
    data = np.moveaxis(data, 0, -1)
    return data


def compute_stimulus_differentiation(stimulus):
    data = reshape_movie(stimulus)
    distances = spectral_differentiation(
        data,
        sample_rate=metadata.TWOP_SAMPLE_RATE,
        window_length=1.0,
        metric='euclidean',
        log_frequency=False,
    )
    return np.median(distances)

### Energy

In [10]:
def spectral_energy_density(stimulus):
    # Remove DC component
    stimulus = stimulus - stimulus.mean()
    # Compute spectral energy density
    spectrum = np.fft.rfft(stimulus, axis=0)
    energy_spectral_density = np.abs(spectrum)**2
    return energy_spectral_density

## Compute

In [11]:
stimulus_properties = pd.DataFrame([
    {
        'stimulus': name,
        'mean luminance': stimulus.mean(),
        'contrast': stimulus.std(),
        'spectral energy': spectral_energy_density(blurred_stimuli[name]).sum(),
        "stimulus differentiation": compute_stimulus_differentiation(blurred_stimuli[name]),
    }
    for name, stimulus in tqdm(stimuli.items()) if stimulus is not None
])

  0%|          | 0/14 [00:00<?, ?it/s]

In [12]:
stimulus_properties['log(stimulus differentiation)'] = np.log10(stimulus_properties['stimulus differentiation'])

In [13]:
stimulus_properties.to_parquet(OUTPUT_DIR/'stimulus_properties.parquet')